## Import Libraries

In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pyperclip

import pandas as pd
import time

### Clipboard copy

In [2]:
def clipboard_input(driver, user_input):
    # save existing clipboard data
    existing_clipboard = pyperclip.paste()

    # copy user_input to clipboard. then paste it
    pyperclip.copy(user_input)
    ActionChains(driver).key_down(Keys.COMMAND).send_keys('v').key_up(Keys.COMMAND).perform()

    # recover the original clipboard data
    pyperclip.copy(existing_clipboard)  
    time.sleep(1)

### Load user info

In [3]:
def load_user_info():
    f = open('user_info.txt', 'r')
    user_info = f.readlines()
    f.close()
    
    return user_info[0][:-1], user_info[1][:-1]

### Naver Login

In [4]:
def login_naver(driver, user_id, user_pw):
    # naver login page
    driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

    # input id 
    driver.find_element_by_xpath('//*[@id="id"]').click()
    clipboard_input(driver, user_id)

    # input pw
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    clipboard_input(driver, user_pw)

    # click login btn
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(1)

    # click 
    driver.find_element_by_xpath('//span[@class="btn_cancel"]').click()

### Enter Details to search in Joonggonara

In [5]:
def get_idx_of_selected_option(option_list, input_text):
    for idx, option in enumerate(option_list):
        if option.text == input_text:
            return idx
    return -1

In [6]:
def enter_details(driver, search_option, item, detail_option):    
    # show 50 items
    driver.find_element_by_xpath('//div[@id="listSizeSelectDiv"]').click()
    time.sleep(1)
    listSize_list = driver.find_elements_by_xpath('//div[@id="listSizeSelectDiv"]/ul/li')
    listSize_list[-1].click()
    time.sleep(1)
    
    # set search options
    driver.find_element_by_xpath('//div[@id="searchOptionSelectDiv"]').click()
    searchBoard_list = driver.find_elements_by_xpath('//div[@id="searchOptionSelectDiv"]/ul/li')
    searchBoard_list[get_idx_of_selected_option(searchBoard_list, search_option['board'])].click()
    time.sleep(1)
    
    driver.find_element_by_xpath('//div[@id="divSearchDateTop"]').click()
    searchDate_list = driver.find_elements_by_xpath('//div[@id="divSearchDateTop"]/ul/li')
    searchDate_list[get_idx_of_selected_option(searchDate_list, search_option['period'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchMenuTop"]').click()
    searchMenu_list = driver.find_elements_by_xpath('//div[@id="divSearchMenuTop"]/ul/li')
    searchMenu_list[get_idx_of_selected_option(searchMenu_list, search_option['menu'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchByTop"]').click()
    searchBy_list = driver.find_elements_by_xpath('//div[@id="divSearchByTop"]/ul/li')
    searchBy_list[get_idx_of_selected_option(searchBy_list, search_option['by'])].click()
    
    # enter item
    driver.find_element_by_xpath('//input[@placeholder="검색어를 입력해주세요"]').send_keys(item)
    
    # click detail search btn
    driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click()

    # enter detail options
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 모두 포함"]').send_keys(detail_option['keywords_and'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 제외"]').send_keys(detail_option['keywords_not'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 중 1개 이상 포함"]').send_keys(detail_option['keywords_or'])
    driver.find_element_by_xpath('//input[@placeholder="다음 어절, 어구 정확히 일치"]').send_keys(detail_option['sentence'])
    
    
    # search
    driver.find_element_by_xpath('//form[@name="frmSearchTop"]/div[@class="input_search_area"]/button[@class="btn-search-green"]').click()
    
    time.sleep(1)

### Get Product Info

In [7]:
def get_post_info(driver, href):
    driver.get(href)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    time.sleep(1)

    soup = bs(driver.page_source, 'html.parser')

    # post_title
    title = soup.select('.title_text')[0].get_text()
    
    # posted_cost
    try:
        cost = soup.select('.cost')[0].get_text()
    except:
        cost = 0

    time.sleep(1)

    return {'cost':cost, 'title' : title}

#### Run ChromeDriver

In [22]:
driver_path = "/Users/dajinhan/Desktop/JOONGGONARA_price_finder/chromedriver"
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(2)

#### Login Naver

In [23]:
user_id, user_pw = load_user_info()
login_naver(driver, user_id, user_pw)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class="btn_cancel"]"}
  (Session info: chrome=86.0.4240.75)


#### Search items from Joonggonara and get boards

In [25]:
# log onto Joonggonara and enter details
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')
time.sleep(1)
driver.switch_to.frame(driver.find_element_by_name("cafe_main"))

search_option={
    'board' : "게시글 전체",
    'period' : "전체기간",
    'menu' : "태블릿 판매/매입", #"주변기기/악세사리",
    'by' : "제목만"
}

item = "아이패드 미니"

detail_option = {
    'keywords_and' : "LTE 5 미개봉",
    'keywords_not' : "중고폰 삽니다 사요 구매 구함 구해요 구합니다",
    'keywords_or' : "", 
    'sentence' : ""
}
# 'keywords_not' : "LTE 중고폰 삽니다 사요 구매 구함 구해요 구합니다"
num_of_items=500

enter_details(driver, search_option, item, detail_option)

In [27]:
board_navigator = driver.find_element_by_xpath('//div[@class="prev-next"]')
board_list = board_navigator.find_elements_by_tag_name('a')
board_href = board_list[0].get_attribute('href')[:-1]
board_idx = 0

#### Get Posts Links

In [28]:
post_key_list=[]
author_list=set([])
while True:
    # change board
    board_idx+=1
    driver.get(board_href+str(board_idx))
    time.sleep(1)
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    
    # get posts in current board
    posts = driver.find_elements_by_css_selector('div.article-board > table > tbody > tr')
    num_of_new_posts = len(posts)
    
    for post in posts:
        
        # get valid posts
        try:
            author = post.find_element_by_class_name('td_name').text.strip()
            href = post.find_element_by_class_name('article').get_attribute('href')
        except:
            num_of_new_posts-=1
            continue
            
        # filter duplicated postings by author
        if author in author_list:
            num_of_new_posts-=1
            continue
        post_key_list.append({"author": author, "href":href})
        author_list.add(author)
        
    # check
    print("read {} pages {} posts. and {} new posts".format(board_idx, len(post_key_list), num_of_new_posts))
    
    # terminate condition
    if num_of_new_posts == 0 or len(post_key_list)>num_of_items:
        break

read 1 pages 7 posts. and 7 new posts
read 2 pages 7 posts. and 0 new posts


#### Get Price Info

In [29]:
price_info=[]
for post_key in post_key_list:
    try:
        post_info = get_post_info(driver, post_key["href"])
        print(post_info['cost'], post_key["author"], post_info['title'])
        price_info.append(post_info)
    except:
        print("cannot attach post")

price_info_pd=pd.DataFrame(price_info)
price_info_pd.to_csv("output/"+item+"_price_info.csv")

570,000원 kimsk59612 
            아이패드 미니5 64GB LTE 미개봉 국내정품 판매합니다.
        
620,000원 즐거운인생시작 
            아이패드미니5 64GB LTE 셀룰러 + 애플펜슬1세대 미개봉 팝니다. 
        
590,000원 에반스7 
            [미개봉] 아이패드 미니5 64기가 실버 셀룰러/lte 미개봉 팝니다.
        
620,000원 중얼대는초이 
            아이패드미니 5 LTE 셀롤러 (미개봉 64g) 팝니다.(사진첨부)
        
560,000원 clubm3647 
            [ 미개봉 신품 ] 아이패드 미니5 64GB LTE 
        
530,000원 모카슈슈 
            아이패드 미니5 64G 셀룰러(LTE) 실버색상 팝니다 (미개봉)
        
790,000원 초사만루 
            미개봉 아이패드 미니 5세대 256GB LTE (골드)
        
